## Boilerplate Code

In [3]:
from sympy import *
import mpmath as mp
import os
from pandas.tseries.offsets import BDay, DateOffset, Timestamp, MonthEnd, MonthBegin
from datetime import date
from scipy.stats import norm
from random import random
from functools import reduce


## Variables

In [16]:
start_date=date(2016,6,18)
number_of_observations=509

sigma=.2
time_step=1/500
drift=.1

file_header = "#\n"
file_header += "# tsd:TimeSeriesData\n"
file_header += "#\n"
file_header += "TBL,tsd:TimeSeriesData\n"
file_header += "HDR,tsd:id,tsd:name,tsd:save_date,tsd:value_date,tsd:value\n"

def tocom_product(start_date, n):
    candidate = start_date + MonthBegin(n-2) + DateOffset(25) - BDay(1)
    if(candidate >= start_date):
        return candidate
    else:
        return start_date + MonthBegin(n-1) + DateOffset(25) - BDay(1)

def eom(x, n):
    return x - DateOffset(1) + MonthEnd(n)

indices=[
    {'name':'sVaR_PLT_DATED_BRENT.USD', 'initial price':40, 'suffix':"lom", 'number of contracts':18, 'precision':3, 'generate contract':eom}, 
    {'name':'sVaR_PLT_DUBAI.USD', 'initial price':35, 'suffix':"lom", 'number of contracts':18, 'precision':3, 'generate contract':eom}, 
    {'name':'sVaR_TOCOM_CRUDE_OIL.JPY', 'initial price':40000, 'suffix':"tocom_crude", 'number of contracts':6, 'precision':0, 'generate contract':eom}, 
    {'name':'sVaR_TOCOM_GASOLINE.JPY', 'initial price':35000, 'suffix':"tocom_product", 'number of contracts':6, 'precision':0, 'generate contract':tocom_product}]

## Functions

In [19]:
def rand_price(prior_price, prec):
    return round(prior_price + prior_price * drift * time_step + sigma * sqrt(time_step) * (random()+random()+random()+random()+random()+random()+random()+random()+random()+random()+random()+random()-6) * prior_price, prec)

def rand_prices(initial_price, n, prec):
    return reduce(lambda x, y: x + [rand_price(x[-1], prec)], list(range(n)), [initial_price])[1:]

def gen_dates(start_date, n):
    return reduce(lambda x, y: x + [x[-1] + BDay(-1)], list(range(n)), [start_date])[1:]

def gen_fwd_rec(id, name, date, price, offset, gen_contract):
    return 'REC,{0},{1},{2},{3},{4}\n'.format(id, name, date.strftime("%Y%m%d"), gen_contract(date, offset).strftime("%Y%m%d"), price)

def gen_fx_rec(id, name, date, price, offset):
    return 'REC,{0},{1},{2},{3},{4}\n'.format(id, name, date.strftime("%Y%m%d"), (date + DateOffset(2 + offset)).strftime("%Y%m%d"), price)

In [20]:
f = open('ts.csv', 'w')
f.write(file_header)

for index in indices:
    for mth in range(1, index['number of contracts'] + 1):
        name = '{0}_@{1}{2}'.format(index['name'], mth, index['suffix'])
        ys = zip(gen_dates(start_date, number_of_observations), rand_prices(index['initial price'], number_of_observations, index['precision']))
        f.write(reduce(lambda x, y: x + gen_fwd_rec(0, name, y[0], y[1], mth, index['generate contract']), ys, ""))

for ccy in [('AUD', 1.39, 3), ('SGD', 1.4, 4), ('MYR', 4.4, 5), ('CNY', 6.89, 6), ('JPY', 103, 0), ('GBP', 0.81, 1), ('EUR', 0.96, 2)]:
    name = '{0}_{1}'.format('sVaR_SPOT_PRICE', ccy[0])
    ys = zip(gen_dates(start_date, number_of_observations), rand_prices(ccy[1], number_of_observations, 3))
    f.write(reduce(lambda x, y: x + gen_fx_rec(0, name, y[0], y[1], ccy[2]), ys, ""))

f.close()